In [5]:
import pandas as pd
import numpy as np
from ast import literal_eval


In [21]:
# Perfrom Extracting
# and convert column tools from string to list
df = pd.read_csv("../jobs.csv", converters={'tools': literal_eval})

In [34]:
# Perfrom Cleaning, Wrangling

# to safely evaluate a string containing a Python literal 

# change year of exp to data type int8
df.loc[:, "year_of_exp"] = df.year_of_exp.astype(np.int8)

# to datetime objects
df.loc[:, "date_time"] = pd.to_datetime(df.date_time, format="%H:%M %d.%m.%Y")

# remove rows where there is no tools and drop duplicates
df[df.tools.notnull]
# df["tools"] = df.tools.drop() 
df.to_csv("tools2.csv")
df.info()
# tools = pd.DataFrame({"tools":df["tools"].agg("sum")})
# tools.value_counts().to_csv("tools.csv")

# df_exploded = df.explode("tools")

# Get unique tools from the exploded DataFrame
# unique_tools = df_exploded["tools"].unique()
# df.info()
# df_exploded

TypeError: Series.notnull() takes 1 positional argument but 2 were given

In [ ]:
s